In [ ]:
##junk

thing_awpp = X_12.loc[X_12['HoA'] == 'away', 'pp%'].copy()
thing_hpp = X_12.loc[X_12['HoA'] == 'home', 'pp%'].copy()

thing_hpp.index = v_minus_one(thing_hpp.index)

thing_hpp.index 



thing_awpp.index = v_one_plus(thing_awpp.index)

thing_awpp.index

thing_awpp.index = v_one_plus(thing_hpp.index)

thing_hpp

In [ ]:
##part 1
##use models with default
##use data set with H/A +1, -1
##do full window for now

##next:
##check if just excluding first 10 days helps (chaotic)
##check if different windows help

##next
## can try tuning (for loops by hand, or ... use grid_search (use ML mastery code))
##-I think tuning will be faster ... just do by hand ... loop over the possible things 
##-ONE for loop over i = (a,b,c,d)... for each model i[0]

##Orrr can try adding features ... here we have to worry about:
##-adding basic features eg pp, and correct fo%
##-scaling numericals
##-dummy vars for categoricals (are there any?) besides H/A
##-num_windows and which lengths for moving avgs
##-filtering the features for increasing complexity inteligently
##-There is a dicotemy: 
##(a)use H/A + numerics or  ... here I think it can be made more like time-series
##(b) just use mumerics (moving avg) ... here I think the order of the games is not important (note Leung did this, and random train)



In [1]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak', 'pts%', 'win%', 'Fclose%', 'PDO'] 

Pischsada features #variation of Ottawa

Goals for Total number of goals scored so far this season

Goals against

Total number of goals conceded so far thisseason

Goals Differential Goals for – Goals against

Power Play Success Rate Ratio – scoring a goal when 5 on 4

Power Kill Success Rate Ratio – not conceding a goal when 4 on 5

Shot % Goal scored/shots takenaaa

Save % Goals conceded/shots saved

Winning Streak Number of consecutive games won

Conference Standing Latest ranking on conference table

Fenwick Close % Possession ratio

PDO Luck parameter

5/5 Goal For/Against Ratio – 5 on 5 Goals For/Against

In [2]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models

##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



##hyper_parameters from here 
##https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
##for xgboost from here 
##https://machinelearningmastery.com/extreme-gradient-boosting-ensemble-in-python/

#xgb

trees = [10, 50, 100, 500, 1000, 5000]  #100  #num of trees
max_depth = range(1,11)  ##3-5
rates = [0.0001, 0.001, 0.01, 0.1, 1.0]  #0.1
subsample in arange(0.1, 1.1, 0.1):  #0.4, 0.5  ##this is 0.1, 0.2 ... 1.0 # % of features to sample


#svc 
kernels in [‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’] #if you use poly, then adjust degree
C in [100, 10, 1.0, 0.1, 0.001]

#gb

learning_rate in [0.001, 0.01, 0.1]
n_estimators [10, 100, 1000]
subsample in [0.5, 0.7, 1.0]
max_depth in [3, 7, 9]


#rfc
max_features [1 to 20]  #key
max_features in [‘sqrt’, ‘log2’]
n_estimators in [10, 100, 1000]

#bc
n_estimators in [10, 100, 1000]

svm_dic = {'kernels':[‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’]}
lrc_dic = {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
lgr_hp_dic = {'solver': [‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’], 'penalty' : [‘none’, ‘l1’, ‘l2’, ‘elasticnet’],
'C' :[100, 10, 1.0, 0.1, 0.01]}

In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


import matplotlib.pyplot as plt
import seaborn as sns


##couple evaluation functions ##removed model_name as variable
def evaluate_binary_classification(y_test, y_pred, y_proba=None, graph = False):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    #try:
    if y_proba != None:
        rocauc_score = roc_auc_score(y_test, y_proba)
    else:
        rocauc_score = "no roc"
    #except: 
    #    pass     
    cm = confusion_matrix(y_test, y_pred)
    if graph == True:
        sns.heatmap(cm, annot=True)
        plt.tight_layout()
        plt.title(f'{model_name}', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
        plt.show()
    print("accuracy: ", accuracy)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1 score: ", f1)
    print("rocauc: ", rocauc_score)
    print(cm)
    #return accuracy, precision, recall, f1, rocauc_score

def evaluate_regression(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print("mae", mae)
    print("mse", mse)
    print('r2', r2)
    
##display null values


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [4]:

data = pd.read_csv("/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_bet_stats_mp.csv")
data.drop(columns=[ 'Unnamed: 0'], inplace=True)

In [5]:
data['won'] = data['won'].apply(int)
data_playoffs = data.loc[data['playoffGame'] == 1, :].copy()  #set aside playoff games ... probably won't use them.
data=  data.loc[data['playoffGame'] == 0, :].copy() 

#sorted(data.columns)

In [6]:
def get_dates(X, season):  #season 20162017 int; returns the list of all dates in chronological order from that season 20161004, 20161005, ...
    X = X.loc[X['season'] == season, :].copy()
    dates_1 = list(set(X.loc[(X['date'] >= 900) & (X['date'] <= 1231) , :]['full_date']))  #2016 part
    dates_2 = list(set(X.loc[(X['date'] >= 100) & (X['date'] <= 800) , :]['full_date']))  #2017 part
    dates = dates_1 + dates_2 #all dates in order
    return dates

#sorted(data.columns)

In [7]:


def make_HA_diff(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    df_model = df_home.sub(df_visitor) 
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)

##try later maye 
def make_HA_concat(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    
    
    df_model = df_home.sub(df_visitor) 
    
    
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)


In [8]:
##have not written yet

def top3_max_val_params(model, X, dates, drop_firs=False):
    pass
    


In [9]:
#this is for  regressors predicting wins - losses, can use this to turn output into win prediction 

def make_win(x):
    if x <= 0:
        return 0
    if x >0:
        return 1

v_make_win = np.vectorize(make_win)

#useage: v_make_win(y_pred)

In [10]:
def regr_model_results(model, model_name, X, dates, step, window_size, prediction_size, drop_first_k_days = 0): #X = data 
    results_dic['model_name'] = []
    results_dic['date'] = []
    results_dic['mae'] = []
    results_dic['mse'] = []
    results_dic['r2'] = []   
    
    
    #drop first k days from dates and X
    dates = dates[drop_first_k_days :]
    X = X.loc[X['full_date'].isin(dates), :].copy()  

    for i in range(step, len(dates), step): ##eg step =10, so 17 rounds
        
        model.fit(X.loc[X['full_date'].isin(dates[max(i-window_size ,0):i]), :],y.loc[y['full_date'].isin(dates[max(i-window_size,0):i]),'goal_difference' ])
        y_pred = model.predict(X.loc[X['full_date'].isin(dates[i:i+prediction_size]), :])
        y_test = y.loc[y['full_date'].isin(dates[i:i+prediction_size]),'goal_difference' ]
        
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results_dic['model_name'].append(model_name)
        results_dic['date'].append(dates[i])
    
        results_dic['mae'].append(mae)
        results_dic['mse'].append(mse)
        results_dic['r2'].append(r2)
        
    return results_dic #!
        
        

In [11]:
def class_model_results(model, model_name, X, dates, step, window_size, prediction_size, drop_first_k_days = 0): #X = data 
    results_dic ={}
    results_dic['model_name'] = []
    results_dic['date'] = []
    results_dic['accuracy'] = []
    results_dic['f1_score'] = []

    #results_dic['precision'] = []
  #  results_dic['recall'] = []
    
    #drop first k days from dates and X
    dates = dates[drop_first_k_days :]
    X = X.loc[X['full_date'].isin(dates), :].copy()  

    for i in range(step, len(dates), step): ##eg step =10, so 17 rounds
        model.fit(X.loc[X['full_date'].isin(dates[max(i-window_size ,0):i]), :],y.loc[y['full_date'].isin(dates[max(i-window_size,0):i]),'won' ])
        y_pred = model.predict(X.loc[X['full_date'].isin(dates[i:i+prediction_size]), :])
        y_test = y.loc[y['full_date'].isin(dates[i:i+prediction_size]),'won' ]
    
        accuracy = accuracy_score(y_test, y_pred)
        #recision = precision_score(y_test, y_pred, zero_division = 0)
        #recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred) #, average = None)
            
        results_dic['model_name'].append(model_name)  #append same model name every iter so same length as others
        results_dic['date'].append(dates[i])
                          
        results_dic['accuracy'].append(accuracy)
        results_dic['f1_score'].append(f1)
        #results_dic['precision'].append(precision)
        #results_dic['recall'].append(recall)
    results_dic['model_name'].append('model_name'+'_avg')  #append same model name every iter so same length as others
    results_dic['date'].append('average')
    results_dic['accuracy'].append(round(np.mean(np.array(results_dic['accuracy'])), 2) ) 
    results_dic['f1_score'].append(round(np.mean(np.array(results_dic['f1_score'])), 2) ) 
    return results_dic #!
        
        

1. two versions of dummies for teams - teams plus H/A and teamH - teamA 
2. play with add/remove when in season (hypothesis: offense early defense favored late) ---> 5 vars for where in year ... 

In [13]:
"dajfdajjj"[:-3]

'dajfda'

In [29]:
def make_diff(statFor, statAway):
    return statFor - statAway

v_make_diff = np.vectorize(make_diff)

def make_per(statFor, statAway):  #example FOWFor/(FOWFor + FOWAgainst)  or ShAt
    try: 
        return statFor/(statFor+statAway)
    except: 
        return 0


v_make_per = np.vectorize(make_per)
 
    
def make_ratio(stat1, stat2):  #example goalsFor/shotsFor = sh%
    try:
        return stat1/stat2
    except:
        return 0
    
v_make_ratio = np.vectorize(make_ratio)

##have to adjust later depending what is convenient to use for stat name ... 

#not using k_days_back .. will use a unioversal fn to just avg past games...
#makes more sense ,,, if they get a 1000 shots against one game, don't want that to leak into 
#other games .. 



def get_per(X, statFor): 
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = statFor[:-3]  #remove last 3
    #statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    X[statFor+'%' ] = v_make_per(X.loc[:,statFor],X.loc[:,statAgainst]) 
    #return v_make_per(X.loc[:,statFor],X.loc[:,statAgainst])


def get_ratio(X, stat1, stat2, new_stat_name):  #stat1/stat2  
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    
    X[new_stat_name] = v_make_ratio(X.loc[:,stat1],X.loc[:,stat2]) 
    #return v_make_ratio(X.loc[:,stat1],X.loc[:,stat2])

def get_diff(X, statFor): 
    #stat_name = goalsFor or faceoffsWonFor  example to keep in mind ... mp style
    ##we do this so we can loop thru existing names in our feature list
    stat_name = statFor[:-3]  #remove last 3
    #statFor = stat_name+'For'
    statAgainst = stat_name+'Against'
    X[stat_name+'Diff']= v_make_diff(X.loc[:,statFor],X.loc[:, statAgainst])
    #return v_make_diff(X.loc[:,statFor],X.loc[:, statAgainst])
    
    

In [15]:
[1,3,4, 7, 8][-3:]

[4, 7, 8]

In [16]:
k = 10

X_16 = data.loc[data['season'] == 20162017, :].copy()
all_dates = sorted(list(X_16['full_date']))

date1 = 20161022
dates_bef_date1 = [date for date in all_dates if date < date1]
dates_bef_date1 = dates_bef_date1[-k :]

stat_name = 'goalsFor'
sk = str(k)
X_16[stat_name+sk+'_avg'] = np.NaN 

X_bef_date1 = X_16.loc[X_16['full_date'].isin(dates_bef_date1), :]
teams_date1 = sorted(list(X_bef_date1['nhl_name']))
nhl_name = 'ANA' 
X = X_16.copy()
k_avg = np.mean(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+sk+'_avg'] =k_avg
k_avg    

3.0

In [17]:
all_dates = sorted(set((X_16['full_date'])))

X_16.head()

,date,season,Open,full_date,nhl_name,game_id,team_id,HoA,won,settled_in,...,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame,goalsFor10_avg
19714,1012,20162017,114,20161012,TOR,2016020001,10,away,0,OT,...,40.626,8.0,0.417,0.302,0.302,2.222,2.149,2.139,0,NaN
19715,1012,20162017,-134,20161012,OTT,2016020001,9,home,1,OT,...,59.050,4.0,0.567,0.221,0.221,2.546,2.711,2.689,0,NaN
19716,1012,20162017,108,20161012,STL,2016020002,19,away,1,REG,...,23.765,10.0,0.310,0.698,0.698,1.394,1.370,1.340,0,NaN
19717,1012,20162017,-128,20161012,CHI,2016020002,16,home,0,REG,...,45.176,13.0,0.569,2.247,2.247,3.791,3.835,3.782,0,NaN
19718,1012,20162017,108,20161012,CGY,2016020003,20,away,0,REG,...,40.285,6.0,0.469,0.514,0.514,3.478,3.496,3.478,0,NaN


In [131]:
 #num of games back max(xurrent - k, 0) = 0 for 1000
     #here X_N = data.iloc[0:date-1, :] all games from all days before present day.  
    #here X = whole season or whatever chumk you want
def get_k_game_avg(X, stat_name,  k_days_back= 10**6 ): #k_days_back #season is random-default
   # for season in season_list:
    #X = data.loc[data['season'] == season, :].copy()      
    #X = data.copy()
    
    if k_days_back < 10**6:
        k = str(k_days_back)
    else: 
        k = "_cumul"
        
    all_dates = sorted(set(X['full_date']))
    X[stat_name+k+'_avg'] = np.NaN  #set up column
    
    #take care of first day of season ... might want to leave it NaN ... nahhh
    
    
    #date0 = all_dates[0]
    #X.loc[(X['full_date'] == date0), stat_name+k+'_avg'] =  X.loc[(X['full_date'] == date0), stat_name]
        
    for date1 in all_dates[1:]: #all but first date so don't get emopty           
        dates_bef_date1 = [date for date in all_dates if date < date1]
        dates_bef_date1 = dates_bef_date1[-k_days_back :] # this grabs all or the last k days of list
        
        X_bef_date1 = X.loc[X['full_date'].isin(dates_bef_date1), :]           
        teams_date1 = sorted(set(X_bef_date1['nhl_name']))
            
            #first take care of all teams not present in the previous days (only a few)
        for nhl_name in set(X['nhl_name']).difference(set(X_bef_date1['nhl_name'])):
            k_avg = X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name] ##should be single number anyway
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_avg'] =k_avg
        
        
        #this is the main step
        for nhl_name in teams_date1:
            k_avg = np.mean(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_avg'] =k_avg
   # return X

In [19]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

In [20]:
feat_Pis_plus = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak', 'pts%', 'win%', 'Fclose%', 'PDO'] 

In [21]:
#features I need to do this ...
feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
#'win_streak_grouped_10', 'conference_standing_grouped_10',
#'PDO'  

#targets 
 #could later try to train classifier to predict this ... tough tho --> reg or no
    

extra = ['corsiPercentage', 
    'penaltiesAgainst',
 'penaltiesFor', 
    'shotAttemptsAgainst',
 'shotAttemptsFor',
    'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor', 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
 'xGoalsPercentage','scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor','blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',

 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',]



In [22]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
###set Leung aside for now ... 
    
feat_Leung = ['ID', 'Date', 'HomeTeam', 'AwayTeam', 'GDiff','GF%', 'CF%','CSh%',
'CSv%', 'FF%','FSh%','FSv%','PDO','PENDiff','ShF%','SDiff','Sh%', 'Sv%','FOW%','W%','FavoritesW%', 'Result']


feat_for_Leung  = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  
extra = ['corsiPercentage', 
    'penaltiesAgainst',
 'penaltiesFor', 
    'shotAttemptsAgainst',
 'shotAttemptsFor',
    'unblockedShotAttemptsAgainst',
 'unblockedShotAttemptsFor', 'savedUnblockedShotAttemptsAgainst',
 'savedUnblockedShotAttemptsFor',
 'xGoalsPercentage','scoreVenueAdjustedxGoalsAgainst',
 'scoreVenueAdjustedxGoalsFor','blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',

 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',]
    
    
feat_plus = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%',  ]


In [24]:
data_Pis = data.loc[:, feat_for_Pis].copy()

In [25]:
data_Pis.head()

,full_date,season,game_id,nhl_name,HoA,opposingTeam,goalsAgainst,goalsFor,powerPlayGoals,powerPlayOpportunities,shotsOnGoalAgainst,shotsOnGoalFor,savedShotsOnGoalAgainst,savedShotsOnGoalFor,fenwickPercentage,won,settled_in
0,20081004,20082009,2008020001,NYR,away,T.B,1.0,2.0,1.0,7.0,21.0,41.0,20.0,39.0,0.6310,1,REG
1,20081004,20082009,2008020001,TBL,home,NYR,2.0,1.0,0.0,4.0,41.0,21.0,39.0,20.0,0.3690,0,REG
2,20081004,20082009,2008020002,PIT,away,OTT,3.0,4.0,0.0,7.0,35.0,30.0,32.0,26.0,0.4778,1,OT
3,20081004,20082009,2008020002,OTT,home,PIT,4.0,3.0,1.0,6.0,30.0,35.0,26.0,32.0,0.5222,0,OT
4,20081005,20082009,2008020003,TBL,away,NYR,2.0,1.0,0.0,5.0,39.0,19.0,37.0,18.0,0.3810,0,REG


In [108]:
#season = 20122013  #select this shortened season because same as Pisch

X_12 = data_Pis.loc[data_Pis['season'] ==20122013, :].copy()


In [109]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goalDiff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 
            'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 


#X_12.head()

In [110]:

feat_for_Pis_small = [ 'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage',] 

get_diff(X_12, 'goalsFor')



In [111]:

get_per(X_12, 'goalsFor')

In [112]:
get_ratio(X_12,  'powerPlayGoals','powerPlayOpportunities', 'pp%')

In [113]:


def one_plus(x):
    return 1+x

v_one_plus = np.vectorize(one_plus)

def minus_one(x):
    return x-1

v_minus_one = np.vectorize(minus_one)

def one_minus(x):
    return 1-x

v_one_minus = np.vectorize(one_minus)

In [114]:
#thing1.apply(one_minus)

X_12['pk%'] = 0


In [115]:
X_12.sort_values(by = ['full_date', 'game_id', 'HoA'], inplace = True)


In [116]:
X_12.reset_index(drop = True, inplace = True)
#X_12['season'].value_counts()

In [ ]:
X_12.loc[X_12['HoA'] == 'away', 'pk%']

In [44]:
v_one_minus(X_12.loc[X_12['HoA'] == 'home' ,['pp%']])[0:5]

array([[1.        ],
       [0.83333333],
       [1.        ],
       [1.        ],
       [0.8       ]])

In [43]:
v_one_minus(X_12.loc[X_12['HoA'] == 'away' ,['pp%']])[0:5]

array([[0.33333333],
       [0.5       ],
       [0.8       ],
       [1.        ],
       [0.6       ]])

In [117]:
#thing1 = X_12.loc[X_12['HoA'] == 'away' ,['pp%']].copy().apply(one_minus).copy()

###??? I have no idea why this is working ... but ok ... 

X_12.loc[X_12['HoA'] == 'home', 'pk%']  = v_one_minus(X_12.loc[X_12['HoA'] == 'away' ,['pp%']].copy())


In [118]:
#thing2  = X_12.loc[X_12['HoA'] == 'home' ,['pp%']].copy().apply(one_minus).copy()

X_12.loc[X_12['HoA'] == 'away',['pk%']] =  v_one_minus(X_12.loc[X_12['HoA'] == 'home' ,['pp%']].copy())



In [81]:
X_12.loc[:, ['pk%']].isnull().sum()
X_12.loc[:10, ['HoA', 'pp%', 'pk%']]

,HoA,pp%,pk%
0,away,0.666667,1.000000
1,home,0.000000,0.333333
2,away,0.500000,0.833333
3,home,0.166667,0.500000
4,away,0.200000,1.000000
5,home,0.000000,0.800000
6,away,0.000000,1.000000
7,home,0.000000,1.000000
8,away,0.400000,0.800000
9,home,0.200000,0.600000


In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 

feat_for_Pis = ['full_date','season', 'game_id', 'nhl_name','HoA','opposingTeam',  'goalsAgainst','goalsFor',
'powerPlayGoals','powerPlayOpportunities',  'shotsOnGoalAgainst','shotsOnGoalFor','savedShotsOnGoalAgainst','savedShotsOnGoalFor', 
    'fenwickPercentage', 'won', 'settled_in',] 
  

In [119]:
get_ratio(X_12, 'goalsFor', 'shotsOnGoalFor', 'sh%')

In [120]:
get_ratio(X_12, 'goalsAgainst', 'shotsOnGoalAgainst', 'sv%')

In [121]:
X_12['sv%'] = one_minus(X_12['sv%'])

In [70]:
X_12.head(2)

,full_date,season,game_id,nhl_name,HoA,opposingTeam,goalsAgainst,goalsFor,powerPlayGoals,powerPlayOpportunities,...,settled_in,goalsDiff,goalsFor%,pp%,pk%,sh%,sv%,ones,ones3000_avg,ones3000_sum
0,20130119,20122013,2012020001,PIT,away,PHI,1.0,3.0,2.0,3.0,...,REG,2.0,0.75,0.666667,1.000000,0.111111,0.962963,1,1.0,NaN
1,20130119,20122013,2012020001,PHI,home,PIT,3.0,1.0,0.0,5.0,...,REG,-2.0,0.25,0.000000,0.333333,0.037037,0.888889,1,1.0,NaN


In [90]:
np.sum([1,2])

3

In [138]:
def get_k_game_sum(X, stat_name, k_days_back= 10**6 ): #k_days_back #season is random-default
   # for season in season_list:
    #X = data.loc[data['season'] == season, :].copy()      
    #X = data.copy()
    if k_days_back < 10**6:
        k = '_'+str(k_days_back)+'_day'
    else: 
        k = "_cumul"
    all_dates = sorted(set(X['full_date']))
    X[stat_name+k+'_sum'] = np.NaN  #set up column
    
    #take care of first day of season ... might want to leave it NaN ... nahhh
    #date0 = all_dates[0]
    #X.loc[(X['full_date'] == date0), stat_name+k+'_sum'] =  X.loc[(X['full_date'] == date0), stat_name]
        
    for date1 in all_dates[1:]: #all but first date so don't get emopty           
        dates_bef_date1 = [date for date in all_dates if date < date1]
        dates_bef_date1 = dates_bef_date1[-k_days_back :] # this grabs all or the last k days of list
        
        X_bef_date1 = X.loc[X['full_date'].isin(dates_bef_date1), :]           
        teams_date1 = sorted(set(X_bef_date1['nhl_name']))
            
            #first take care of all teams not present in the previous days (only a few)
        for nhl_name in set(X['nhl_name']).difference(set(X_bef_date1['nhl_name'])):
            k_sum = X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name] ##should be single number anyway
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_sum'] =k_sum
        
        
        #this is the main step
        for nhl_name in teams_date1:
            k_sum = np.sum(X_bef_date1.loc[(X_bef_date1['nhl_name'] == nhl_name) , stat_name])
            X.loc[(X['full_date'] == date1)& (X['nhl_name'] == nhl_name) , stat_name+k+'_sum'] =k_sum
   # return X

In [122]:
X_12['ones'] = 1

In [123]:
get_k_game_sum(X_12, 'won') 

In [124]:
get_k_game_sum(X_12, 'ones')

In [125]:
X_12.rename(columns = {'ones_cumul_sum': 'games_so_far'}, inplace = True)

In [126]:
X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'sh%', 'sv%', 'ones',
       'won_cumul_sum', 'games_so_far'],
      dtype='object')

In [130]:
X_12.loc[200:230, ['full_date','HoA','nhl_name', 'won_cumul_sum', 'games_so_far', 'win%']]

,full_date,HoA,nhl_name,won_cumul_sum,games_so_far,win%
200,20130201,away,PHI,2.0,7.0,0.285714
201,20130201,home,WSH,1.0,7.0,0.142857
202,20130201,away,OTT,5.0,7.0,0.714286
203,20130201,home,CAR,2.0,5.0,0.400000
204,20130201,away,WPG,3.0,7.0,0.428571
205,20130201,home,TBL,5.0,6.0,0.833333
206,20130201,away,STL,6.0,7.0,0.857143
207,20130201,home,DET,3.0,6.0,0.500000
208,20130201,away,ARI,2.0,7.0,0.285714
209,20130201,home,DAL,2.0,7.0,0.285714


In [141]:
X_12['PDO'] = X_12['sh%'] + X_12['sv%']

In [102]:
##I am stuck on this win% thing ... let me try a model ... for now

X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'ones', 'onescumul_avg',
       'games_so_far', 'sh%', 'sv%', 'PDO'],
      dtype='object')

In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 


In [129]:
get_ratio(X_12, 'won_cumul_sum', 'games_so_far', 'win%')

In [134]:
get_k_game_sum(X_12, 'won', k_days_back=10)

In [135]:
get_k_game_sum(X_12, 'ones', k_days_back=10)

In [136]:
X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'sh%', 'sv%', 'ones',
       'won_cumul_sum', 'games_so_far', 'win%', 'won_10day_sum',
       'ones_10day_sum'],
      dtype='object')

In [137]:
get_ratio(X_12, 'won_10day_sum','ones_10day_sum','last_10_games_win%')

In [139]:
X_12.head()

,full_date,season,game_id,nhl_name,HoA,opposingTeam,goalsAgainst,goalsFor,powerPlayGoals,powerPlayOpportunities,...,pk%,sh%,sv%,ones,won_cumul_sum,games_so_far,win%,won_10day_sum,ones_10day_sum,last_10_games_win%
0,20130119,20122013,2012020001,PIT,away,PHI,1.0,3.0,2.0,3.0,...,1.000000,0.111111,0.962963,1,1.0,1.0,1.0,1.0,1.0,1.0
1,20130119,20122013,2012020001,PHI,home,PIT,3.0,1.0,0.0,5.0,...,0.333333,0.037037,0.888889,1,0.0,1.0,0.0,0.0,1.0,0.0
2,20130119,20122013,2012020002,OTT,away,WPG,1.0,4.0,1.0,2.0,...,0.833333,0.108108,0.964286,1,1.0,1.0,1.0,1.0,1.0,1.0
3,20130119,20122013,2012020002,WPG,home,OTT,4.0,1.0,1.0,6.0,...,0.500000,0.035714,0.891892,1,0.0,1.0,0.0,0.0,1.0,0.0
4,20130119,20122013,2012020003,CHI,away,L.A,2.0,5.0,1.0,5.0,...,1.000000,0.227273,0.904762,1,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
feat_Pis = ['goalsAgainst', 'goalsFor', 'goal_diff', 'goal_perc', 'PP%', 'PK%', 'sh%', 'sv%', 'win_streak_grouped_10', 'conference_standing_grouped_10','Fclose%', 'PDO'] 


In [161]:
feat_Pis_to_sum  = ['goalsAgainst', 'goalsFor', 'goalsDiff']
feat_Pis_to_avg = ['goalsFor%',  'pp%', 'pk%',  'sh%', 'sv%', 'fenwickPercentage', 'PDO',    ]  

##NOTE! This is a bit inaccurate to average these ... one should find cumul_sums then do the % calculations above for accurate %
##let's see if they are that different ... 

#'last_10_games_win%',  'win%',     omtted because already a 10 gm avg, 

for feat_to_sum in feat_Pis_to_sum:
    get_k_game_sum(X_12, feat_to_sum)
    
    

In [162]:
for feat_to_avg in feat_Pis_to_avg:
    get_k_game_avg(X_12, feat_to_avg)

In [163]:
X_12.head(3)

,full_date,season,game_id,nhl_name,HoA,opposingTeam,goalsAgainst,goalsFor,powerPlayGoals,powerPlayOpportunities,...,goalsAgainst_cumul_sum,goalsFor_cumul_sum,goalsDiff_cumul_sum,goalsFor%_cumul_avg,pp%_cumul_avg,pk%_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage_cumul_avg,PDO_cumul_avg
0,20130119,20122013,2012020001,PIT,away,PHI,1.0,3.0,2.0,3.0,...,1.0,3.0,2.0,0.75,0.666667,1.000000,0.111111,0.962963,0.4940,1.074074
1,20130119,20122013,2012020001,PHI,home,PIT,3.0,1.0,0.0,5.0,...,3.0,1.0,-2.0,0.25,0.000000,0.333333,0.037037,0.888889,0.5060,0.925926
2,20130119,20122013,2012020002,OTT,away,WPG,1.0,4.0,1.0,2.0,...,1.0,4.0,3.0,0.80,0.500000,0.833333,0.108108,0.964286,0.5368,1.072394


In [164]:
X_12.columns

Index(['full_date', 'season', 'game_id', 'nhl_name', 'HoA', 'opposingTeam',
       'goalsAgainst', 'goalsFor', 'powerPlayGoals', 'powerPlayOpportunities',
       'shotsOnGoalAgainst', 'shotsOnGoalFor', 'savedShotsOnGoalAgainst',
       'savedShotsOnGoalFor', 'fenwickPercentage', 'won', 'settled_in',
       'goalsDiff', 'goalsFor%', 'pp%', 'pk%', 'sh%', 'sv%', 'ones',
       'won_cumul_sum', 'games_so_far', 'win%', 'won_10day_sum',
       'ones_10day_sum', 'last_10_games_win%', 'PDO', 'goalsAgainst_cumul_sum',
       'goalsFor_cumul_sum', 'goalsDiff_cumul_sum', 'goalsFor%_cumul_avg',
       'pp%_cumul_avg', 'pk%_cumul_avg', 'sh%_cumul_avg', 'sv%_cumul_avg',
       'fenwickPercentage_cumul_avg', 'PDO_cumul_avg'],
      dtype='object')

In [168]:
data_Pis = X_12.loc[:, ['HoA', 'goalsAgainst_cumul_sum',
       'goalsFor_cumul_sum', 'goalsDiff_cumul_sum', 'goalsFor%_cumul_avg',
       'pp%_cumul_avg', 'pk%_cumul_avg', 'sh%_cumul_avg', 'sv%_cumul_avg',
       'fenwickPercentage_cumul_avg', 'PDO_cumul_avg','last_10_games_win%',   'win%',  ]]

In [169]:
data_Pis.head(3)

,HoA,goalsAgainst_cumul_sum,goalsFor_cumul_sum,goalsDiff_cumul_sum,goalsFor%_cumul_avg,pp%_cumul_avg,pk%_cumul_avg,sh%_cumul_avg,sv%_cumul_avg,fenwickPercentage_cumul_avg,PDO_cumul_avg,last_10_games_win%,win%
0,away,1.0,3.0,2.0,0.75,0.666667,1.000000,0.111111,0.962963,0.4940,1.074074,1.0,1.0
1,home,3.0,1.0,-2.0,0.25,0.000000,0.333333,0.037037,0.888889,0.5060,0.925926,0.0,0.0
2,away,1.0,4.0,3.0,0.80,0.500000,0.833333,0.108108,0.964286,0.5368,1.072394,1.0,1.0


In [156]:
dummies_pm1_Pis, y_Pis  = make_HA_diff(data, season = 20122013)

In [158]:
dummies_pm1_Pis = dummies_pm1_Pis.iloc[:, :30].copy() 
dummies_pm1_Pis

,ANA,ARI,BOS,BUF,CAR,CBJ,CGY,CHI,COL,DAL,...,OTT,PHI,PIT,SJS,STL,TBL,TOR,VAN,WPG,WSH
0,0,0,0,0,0,0,0,0,0,0,...,0,-1,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,-1,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0,0,0,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
716,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
717,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
def make_HA_diff(X, season, list_var_names = None ):
    X = X.loc[X['season'] == season, :].copy()
    X_H = X.loc[X['HoA'] == 'home',:].copy()
    X_A = X.loc[X['HoA'] == 'away',:].copy()
    X_H['goal_difference'] = X_H['goalsFor'] - X_H['goalsAgainst']  ##note every thing is based in home data
    X_H.reset_index(drop = True, inplace = True)
    X_A.reset_index(drop = True, inplace = True)
    df_visitor = pd.get_dummies(X_H['nhl_name'], dtype=np.int64)
    df_home = pd.get_dummies(X_A['nhl_name'], dtype=np.int64)
    df_model = df_home.sub(df_visitor) 
    df_model['date'] = X_H['date']
    df_model['full_date'] = X_H['full_date']
    
    df_model['game_id'] = X_H['game_id']
    df_model['home_id'] = X_H['team_id']
    df_model['away_id'] = X_A['team_id'] 
    y = X_H.loc[:,['date', 'full_date','game_id', 'Open','goal_difference', 'won']].copy()   ##these are from home team perspective; 'Open' is for betting 
    return (df_model, y)


In [ ]:
data_Pis = pd.concat(dummies_pm1_Pis,  axis =1)